In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, normal, choice
import os, shutil

# Specify parameter sampler
ps = RandomParameterSampling( {
        '--C': uniform(0.5, 1),
        '--max_iter': choice(20, 40, 60, 80, 100)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, delay_evaluation=5, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

shutil.copy('train.py', 'training')

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory="./",
                      entry_script='train.py',
                      compute_target=compute_target)
               
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=100,
                                     max_concurrent_runs=4)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(config=hyperdrive_config)

RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run)

best_run_metrics = best_run.get_metrics()
print(best_run_metrics)

In [ ]:
print(best_run.get_file_names())

In [ ]:
# Register the model
run.register_model(model_path='outputs/hyperdrive_model.joblib', model_name='hyperdrive_model',
                   tags={'Training context':'Parameterized SKLearn Estimator'})

In [ ]:
model = best_run.register_model(model_name='hyperdrive_model', model_path='ooutputs/hyperdrive_model.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data_url = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
ds = TabularDatasetFactory.from_delimited_files(path=data_url)

In [ ]:
from train import clean_data

x, y = clean_data(ds)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
train_data = x_train.join(y_train)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    debug_log='automl_errors.log',
    compute_target=compute_target,
    training_data=train_data,
    label_column_name='y',
    n_cross_validations= 5,
    primary_metric='Accuracy',
   )

In [2]:
# Submit your automl run

### YOUR CODE HERE ###
remote_run = experiment.submit(automl_config, show_output = True)

# remote_run = experiment.submit(automl_config, show_output = False)

In [ ]:
from azureml.core.experiment import Experiment
experiment = Experiment(ws, 'automl_remote')
remote_run = experiment.submit(automl_config, show_output=True)

In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###

best_run, fitted_model = automl_run.get_output()
print(best_run)
print(fitted_model)

In [ ]:
best_run.get_tags()

In [ ]:
# Registering the model

model = best_run.register_model(model_name="best_run", model_path="outputs/model.joblib", tags={"accuracy" : <your_accuracy>}, description="<your_description>")